In [4]:
!pip install -U spacy
!pip install keras
!python -m spacy download en
!python -m spacy download en_core_web_lg

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [2]:
!pip install hyperas
!pip install hyperopt

In [0]:
import spacy
import en_core_web_lg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

from gensim.parsing.preprocessing import preprocess_documents

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [0]:
#load the pretrained CNN vectorizer from spacy
nlp = en_core_web_lg.load()

In [0]:
tokenizer = nlp.Defaults.create_tokenizer(nlp)
texts = ['a','b we! #1234 r a', 'dkkwerk weoiria fads']
doc = nlp("Hello I am Testing what is going on here?")
print(doc.vector)

## Data Collation

In [33]:
#read in data, only using 1m rows due to hardware/time constraints (normal size 3.6m)
train_df = pd.read_csv("https://mt-proj-001.s3.us-east-2.amazonaws.com/train.csv", header=None, nrows=1000000)
train_df = train_df.rename(columns={0:'label',2:'review'}).drop(columns=[1])
train_df.head()

,label,review
0,2,This sound track was beautiful! It paints the ...
1,2,I'm reading a lot of reviews saying that this ...
2,2,This soundtrack is my favorite music of all ti...
3,2,I truly like this soundtrack and I enjoy video...
4,2,"If you've played the game, you know how divine..."


In [34]:
test_df = pd.read_csv("https://mt-proj-001.s3.us-east-2.amazonaws.com/test.csv", header=None, nrows=108000)
test_df = test_df.rename(columns={0:'label',2:'review'}).drop(columns=[1])
test_df.head()

,label,review
0,2,My lovely Pat has one of the GREAT voices of h...
1,2,Despite the fact that I have only played a sma...
2,1,I bought this charger in Jul 2003 and it worke...
3,2,Check out Maha Energy's website. Their Powerex...
4,2,Reviewed quite a bit of the combo players and ...


In [35]:
#pull out the review labelling into a Series
test_labels = test_df['label'].replace(1,0).replace(2,1)
test_labels.head()

0    1
1    1
2    0
3    1
4    1
Name: label, dtype: int64

In [36]:
#pull out the review labelling into a Series
train_labels = train_df['label'].replace(1,0).replace(2,1)
train_labels.head()

0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64

In [0]:
#create a generator for the list of vectors for each review text
train_tokens = []
for doc in nlp.pipe(train_df['review'], disable=["tagger","parser","ner"]):
  train_tokens.append(doc.vector) 

test_tokens = []
for doc in nlp.pipe(test_df['review'], disable=["tagger","parser","ner"]):
  test_tokens.append(doc.vector)  

In [0]:
X_train = train_tokens
X_test = test_tokens
y_train = train_labels
y_test = test_labels

## Untuned DNN Keras Model

In [0]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

In [0]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [0]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [42]:
X_train_scaled.shape

(1000000, 300)

In [0]:
import pickle

with open('X_train_scaled.pickle', 'wb') as handle:
    pickle.dump(X_train_scaled, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('X_test_scaled.pickle', 'wb') as handle:
    pickle.dump(X_test_scaled, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('y_train_categorical.pickle', 'wb') as handle:
    pickle.dump(y_train_categorical, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('y_test_categorical.pickle', 'wb') as handle:
    pickle.dump(y_test_categorical, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

number_hidden_nodes = 100
input_dim = X_train_scaled.shape[1]

model = Sequential()
model.add(Dense(units=number_hidden_nodes, activation='relu', input_dim=input_dim))
model.add(Dense(units=number_hidden_nodes, activation='relu'))
model.add(Dense(units=2, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 40,402
Trainable params: 40,402
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 1000000 samples
Epoch 1/100
1000000/1000000 - 46s - loss: 0.3506 - acc: 0.8463
Epoch 2/100
1000000/1000000 - 46s - loss: 0.3293 - acc: 0.8573
Epoch 3/100
1000000/1000000 - 46s - loss: 0.3212 - acc: 0.8610
Epoch 4/100
1000000/1000000 - 46s - loss: 0.3159 - acc: 0.8635
Epoch 5/100
1000000/1000000 - 45s - loss: 0.3123 - acc: 0.8650
Epoch 6/100
1000000/1000000 - 45s - loss: 0.3091 - acc: 0.8668
Epoch 7/100
1000000/1000000 - 46s - loss: 0.3069 - acc: 0.8676
Epoch 8/100
1000000/1000000 - 45s - loss: 0.3050 - acc: 0.8686
Epoch 9/100
1000000/1000000 - 46s - loss: 0.3031 - acc: 0.8694
Epoch 10/100
1000000/1000000 - 46s - loss: 0.3016 - acc: 0.8700
Epoch 11/100
1000000/1000000 - 46s - loss: 0.3001 - acc: 0.8710
Epoch 12/100
1000000/1000000 - 45s - loss: 0.2987 - acc: 0.8714
Epoch 13/100
1000000/1000000 - 46s - loss: 0.2976 - acc: 0.8719
Epoch 14/100
1000000/1000000 - 46s - loss: 0.2967 - acc: 0.8723
Epoch 15/100
1000000/1000000 - 46s - loss: 0.2958 - acc: 0.8728
Epoch 16/100
1000000/100

In [45]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

108000/108000 - 2s - loss: 0.3489 - acc: 0.8579
Neural Network - Loss: 0.34887827003333305, Accuracy: 0.8579074144363403


In [0]:
model.save("Untuned_DNN.h5")

##Hyperas

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
# Hyperas requires a file copy of the notebook to be mounted locally in ipynb format.
# Copy/download the file
fid = drive.ListFile({'q':"title='SpaCy_Word2Vec.ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('SpaCy_Word2Vec.ipynb')

In [62]:
from hyperas.distributions import uniform
from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras import backend as K
from keras.utils import print_summary

def data():
  import pickle
  with open('X_train_scaled.pickle', 'rb') as handle:
    x_train = pickle.load(handle)
  with open('X_test_scaled.pickle', 'rb') as handle:
    x_test = pickle.load(handle)
  with open('y_train_categorical.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
  with open('y_test_categorical.pickle', 'rb') as handle:
    y_test = pickle.load(handle)

  return x_train, y_train, x_test, y_test 

def create_model(x_train, y_train, x_test, y_test):
    print(x_train.shape)
    model= Sequential() 
    model.add(Dense({{choice([50,200,500])}}, input_dim=x_train.shape[1], activation= 'relu'))
#     model.add(Activation('relu'))
    model.add(Dropout({{uniform(0,1)}}))
    model.add(Dense({{choice([50,200,500])}},activation= 'relu'))
    #model.add(Activation('relu'))
    model.add(Dropout({{uniform(0,1)}}))
    model.add(Dense(2, activation= 'softmax'))
   # model.add(Activation('linear'))

    model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])

    print_summary(model, line_length=None, positions=None, print_fn=None)

    result= model.fit(x_train, y_train,
                      batch_size={{choice([64,128])}},
                      epochs={{choice([50,100,150])}},
                      verbose=2,
                      validation_split =0.15)
    validation_acc= np.min(result.history['val_loss'])
    print('Lowest Validation Loss:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}   

best_run, best_model= optim.minimize(model=create_model,
                                     data=data,
                                     algo=tpe.suggest,
                                     max_evals=5,
                                     trials=Trials(),
                                     notebook_name='SpaCy_Word2Vec')

>>> Imports:
#coding=utf-8

try:
    import spacy
except:
    pass

try:
    import en_core_web_lg
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import sklearn
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from gensim.parsing.preprocessing import preprocess_documents
except:
    pass

try:
    from tensorflow.keras.utils import to_categorical
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    from tensorflow.keras.utils import to_categorical
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

try:

In [65]:
import pickle
with open('X_test_scaled.pickle', 'rb') as handle:
    x_test = pickle.load(handle)
with open('y_test_categorical.pickle', 'rb') as handle:
    y_test = pickle.load(handle)

print("Evalutation of best performing model:")
print(best_model.evaluate(x_test,y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

best_model.save('Hyperas_tuned_DNN.h5')

Evalutation of best performing model:
108000/108000 [==============================] - 2s 19us/step
[0.6724774932861328, 0.6613055555555556]
Best performing model chosen hyper-parameters:
{'Dense': 0, 'Dense_1': 2, 'Dropout': 0.9758185183456943, 'Dropout_1': 0.288662535902546, 'batch_size': 0, 'epochs': 1}


In [0]:
from google.colab import files
files.download('Hyperas_tuned_DNN.h5')


In [67]:
model_loss, model_accuracy = best_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Neural Network - Loss: 0.6724774932861328, Accuracy: 0.6613055555555556


## Untuned Keras w LSTM

In [0]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import math
import numpy as np

def data(proportion=1.0):
  '''Returns a proportion of the rows from the csv files in a scaled train/test split'''
  nlp = en_core_web_lg.load()

  train_df = pd.read_csv("https://mt-proj-001.s3.us-east-2.amazonaws.com/train.csv", 
                         header=None, nrows=math.floor(3600000 * proportion))
  train_df = train_df.rename(columns={0:'label',2:'review'}).drop(columns=[1])


  test_df = pd.read_csv("https://mt-proj-001.s3.us-east-2.amazonaws.com/test.csv", 
                        header=None, nrows=math.floor(400000 * proportion))
  test_df = test_df.rename(columns={0:'label',2:'review'}).drop(columns=[1])


  #pull out the review labelling into a Series
  test_labels = test_df['label'].replace(1,0).replace(2,1)
  
  train_labels = train_df['label'].replace(1,0).replace(2,1)

  print("Beginning tokenizing...")
  #create a generator for the list of vectors for each review text
  train_tokens = []
  for doc in nlp.pipe(train_df['review'], disable=["tagger","parser","ner"]):
    token_vecs = []
    for token in doc:
      token_vecs.append(token.vector)
    train_tokens.append(token_vecs) 

  test_tokens = []
  for doc in nlp.pipe(test_df['review'], disable=["tagger","parser","ner"]):
    token_vecs = []
    for token in doc:
      token_vecs.append(token.vector)
    test_tokens.append(token_vecs)  

  print("Padding and truncating...")
  #for each set of vectors, find the median length, and truncate/pad each set to the median length
  #yes this is very slow
  PAD_VECTOR = [0.0] * 300
  length_list = []
  for vector_list in train_tokens:
    length_list.append(len(vector_list))
  for vector_list in test_tokens:
    length_list.append(len(vector_list))

  median_len = int(np.median(length_list))
  print(f"Median length of a review is: {median_len}")

  for i in range(len(train_tokens)):
    if len(train_tokens[i]) > median_len:
      train_tokens[i] = train_tokens[i][:median_len]
    elif len(train_tokens[i]) < median_len:
      train_tokens[i] += (median_len - len(train_tokens[i])) * [PAD_VECTOR]
    if len(train_tokens[i]) != median_len:
      print('oops', i)
  for i in range(len(test_tokens)):
    if len(test_tokens[i]) > median_len:
      test_tokens[i] = test_tokens[i][:median_len]
    elif len(test_tokens[i]) < median_len:
      test_tokens[i] += (median_len - len(test_tokens[i])) * [PAD_VECTOR]

  #debug prints
  print(len(train_tokens[0]))
  print(len(train_tokens[0][0]))
  print(type(train_tokens))
  print(type(train_tokens[0]))
  print(type(train_tokens[0][0]))

  #variable reassignment
  X_train = np.array(train_tokens)
  X_test = np.array(test_tokens)
  y_train = train_labels
  y_test = test_labels

  print(f"Train token shape: {np.shape(X_train)} \n Train label shape: {np.shape(y_train)}")
  print("Scaling, about to return...")
  # Create a StandardScater model and fit it to the training data
  # reshaping due to the data being 3D
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
  X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
  #X_scaler = StandardScaler().fit(X_train)

  #X_train_scaled = X_scaler.transform(X_train)
  #X_test_scaled = X_scaler.transform(X_test)


  y_train_categorical = to_categorical(y_train)
  y_test_categorical = to_categorical(y_test)

  return X_train_scaled, y_train_categorical, X_test_scaled, y_test_categorical

In [10]:
PROPORTION = 0.007
X_train_scaled, y_train_categorical, X_test_scaled, y_test_categorical = data(PROPORTION)

Beginning tokenizing...
Padding and truncating...
Median length of a review is: 77
77
300
<class 'list'>
<class 'list'>
<class 'numpy.ndarray'>
Train token shape: (25200, 77, 300) 
 Train label shape: (25200,)
Scaling, about to return...


In [11]:
print(X_train_scaled.shape, X_test_scaled.shape, y_train_categorical.shape, y_test_categorical.shape)

(25200, 77, 300) (2800, 77, 300) (25200, 2) (2800, 2)


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

neurons = 20
#input_dim = X_train_scaled.shape[1]
batch_size = 100
time_step = 77
data_dim = 300

model = Sequential()
#model.add(Dense(units=neurons, activation='relu', input_dim=input_dim))
model.add(LSTM(neurons, dropout=0.1, recurrent_dropout=0.1, stateful=True, return_sequences=True, batch_input_shape=(batch_size, time_step, data_dim)))
model.add(LSTM(neurons, dropout=0.1, recurrent_dropout=0.1, stateful=True))
model.add(Dense(units=2, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (100, 77, 20)             25680     
_________________________________________________________________
lstm_3 (LSTM)                (100, 20)                 3280      
_________________________________________________________________
dense_1 (Dense)              (100, 2)                  42        
Total params: 29,002
Trainable params: 29,002
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [0]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2,
    validation_split =0.15
)

Train on 21420 samples, validate on 3780 samples
Epoch 1/100
